# Planoconvex lens (sphere)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchlensmaker as tlm

In [ ]:
lens_diameter = 60. # lens diameter in mm

class Optics(tlm.Module):
    def __init__(self):
        super().__init__()
        
        self.shape = tlm.CircularArc(lens_diameter, nn.Parameter(torch.tensor(-200.)))

        self.lens = tlm.PlanoLens(self.shape, (1.0, 1.5), outer_thickness=2.0)

        self.optics = tlm.OpticalSequence(
            tlm.PointSourceAtInfinity(beam_diameter=0.9*lens_diameter),
            tlm.Gap(15.),
            
            self.lens,
            
            tlm.Gap(100.0), # focal length
            tlm.FocalPoint(),
        )

    def forward(self, inputs, sampling):
        return self.optics(inputs, sampling)

optics = Optics()


tlm.render_plt(optics)

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    sampling = {"rays": 10},
    num_iter = 250
)

tlm.render_plt(optics)

In [ ]:
part = tlm.lens_to_part(optics.lens)
part